# Benchmark wav2vec2 performance in onnx

In [ ]:
# install librosa for getting duration of audio files
!pip install librosa

In [6]:
!pip uninstall  onnxruntime-gpu -y
!pip install onnxruntime

Found existing installation: onnxruntime-gpu 1.10.0
Uninstalling onnxruntime-gpu-1.10.0:
  Successfully uninstalled onnxruntime-gpu-1.10.0
  Using cached onnxruntime-1.10.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.9 MB)


In [4]:
# create onnx modelsf
from convert_wav2vec2 import convert_wav2vec2_onnx

!rm -rf ./exports

model_id = "facebook/wav2vec2-base-960h"
convert_wav2vec2_onnx(model_id=model_id, optimize=True, quantize=True)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


save model at: /home/ubuntu/onnx-transformers/speech/exports/wav2vec2-base-960h.onnx
input_names:  ['input_values']
current input shape {'input_values': torch.Size([1, 219040])}
Using framework PyTorch: 1.10.0+cu102
output_names:  ['logits']
dynamic_axes:  {'input_values': {0: 'batch_size', 1: 'sequence'}, 'logits': {0: 'batch_size', 1: 'sequence'}}


/home/ubuntu/.local/lib/python3.8/site-packages/torch/onnx/utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTorch release. The code will work as it is False if models are not larger than 2GB, Otherwise set to False because of size limits imposed by Protocol Buffers.
  warnings.warn("`use_external_data_format' is deprecated and ignored. Will be removed in next "
/home/ubuntu/.local/lib/python3.8/site-packages/torch/nn/functional.py:2359: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  _verify_batch_size([input.size(0) * input.size(1) // num_groups, num_groups] + list(input.size()[2:]))
/home/ubuntu/.local/

{'EmbedLayerNormalization': 0, 'Attention': 12, 'Gelu': 8, 'FastGelu': 0, 'BiasGelu': 12, 'LayerNormalization': 1, 'SkipLayerNormalization': 25}
optimized model saved at: /home/ubuntu/onnx-transformers/speech/exports/wav2vec2-base-960h_self.onnx


quantized model saved at: /home/ubuntu/onnx-transformers/speech/exports/wav2vec2-base-960h-q8.onnx
outpus are different


# Run ONNX CPU Predictions

In [4]:
import onnxruntime as ort
import numpy as np 
from transformers import Wav2Vec2Processor
from convert_wav2vec2 import get_sample,get_inuputs_from_audio
import time 
import librosa

sess_options = ort.SessionOptions()

# Set graph optimization level
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL

model_id = "facebook/wav2vec2-base-960h"

model_path="exports/wav2vec2-base-960h-q8.onnx"
ort_session = ort.InferenceSession(model_path, sess_options)
processor = Wav2Vec2Processor.from_pretrained(model_id)


def onnx_asr(path,sess,processor):
    # get inputs
    onnx_inputs = get_inuputs_from_audio(path=path, processor=processor, tensor_type="np")
    # run inference
    st = time.time()
    logits = sess.run(None, onnx_inputs.data)[0]
    dur = time.time() - st
    # decode
    predicted_ids=np.argmax(logits, axis=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription, dur


In [5]:
print(f"running onnx inference with {model_id} on CPU")
sample = get_sample(1)
sample = "yt.mp3"
print(f"Duration of {sample} is {librosa.get_duration(filename=sample)}s")

trans , duration = onnx_asr("yt.mp3",ort_session,processor)
print(f"Prediction with ORT took {round(duration,2)}s")
print(f"Meaining: 1 second audio takes {round(round(duration,2)/librosa.get_duration(filename=sample),2)} seconds to predict")
print(f"transcript: \n{trans}")

running onnx inference with facebook/wav2vec2-base-960h on CPU
Duration of yt.mp3 is 192.0s
Prediction with ORT took 52.54s
Meaining: 1 second audio takes 0.27 seconds to predict
transcript: 
O EVERY ONE ON NAT HUD PACE  TO DAY ER IN A FINE TUN OVISION TRANSFORMER FOR IN AGE FLATATION ESN P  FOCH PUT TOGETHER OMAKE A SUPER EA  TRAIN OVISION TRANSFORMER OR ANYTHING YOUSN PICTES ON THE WAPASTEDLY ALL YEU TO DUSE THO FIN THE PLASES YOU LO OE IN AGE FOTS FARE FOR AND THAT WE USE AND INS SEARCHA YA  FLECT IN AGES OF THOSE  LA S  AT TO THAT IL TRAIN OF THISION TRANSFORMER A THE IMAGES WE CLC  WE SHOP O THE PHUTN PACE UP  SO ANY ONE AROUD THE WORLD A TRIU RH  TO GOSTARTED YOU GO THE HUNN PICH WE GON GETOD PIT OPEN IN COLAT TACTES  DAM AS A PRELIMINARY STEP WOLN STALL THE REPAR T ANS PWON STATS TAN PLY WER THREE TO DEFINE OUR SEARCH TURMS TAKE E A TE PASSES THAT WE MON FINTION AR MOL PREDICT WHAT' SAY WE WANT THA GADOG RE PA AC WE SIPLY RITE THE NAME TO THE H TE DOG BREES ONT TACT PLOTERS TE R

In [3]:
print(f"running onnx inference with {model_id} on CPU")
sample = get_sample(1)
sample = "yt.mp3"
print(f"Duration of {sample} is {librosa.get_duration(filename=sample)}s")

trans , duration = onnx_asr("yt.mp3",ort_session,processor)
print(f"Prediction with ORT took {round(duration,2)}s")
print(f"Meaining: 1 second audio takes {round(round(duration,2)/librosa.get_duration(filename=sample),2)} seconds to predict")
print(f"transcript: \n{trans}")

running onnx inference with facebook/wav2vec2-base-960h on CPU
Duration of yt.mp3 is 192.0s
Prediction with ORT took 52.71s
Meaining: 1 second audio takes 0.27 seconds to predict
transcript: 
AH HA EVERY ONE I'M NATE FROM HUGGIN FACE AND TO DAY WE'RE IN A FINE TUNE OF VISION TRANSFORMER FOR IMAGE CLASSIFICATION USING HUGGAN PIKS A PROJECT WE PUT TOGETHER TO MAKE IT SUPEREASY TO TRAN OF VISION TRANSFORMER FOR ANYTHING USING PICTURES ON THE WEB BASICALLY ALL YOU'VE TO DOSE TO FINE THE CLASSES YOUD LIKE TO BUILD AN IMAGE CLASS FIRE FOR AND THE WE'LL USE AN IMAGE SEARCH A P A TO COLLECT IMAGES OF THOSE DESIRED CLASSES AFTER THAT WE'LL TRAIN A VISION TRANSFORMER ON THE IMAGES WE COLLECT AND PUSH IT UP TO THE HUGAN FACE HUB SO ANY ONE AROUND THE WORLD CAN TRY OUT YOUR CREATION  TO GET STARTED YOUAN GO TO THE HUGAN PICKS REEPO AND GET HUB AND CLICK OPENING COALAB TO ACCES THE DEMO AS A PRELIMINARY STEP WE'LL INSTALL THE REQUIRED DO PENDANCIES ONCE THAT'S COMPLETE WE'RE FREE TO DEFINE OUR SEAR

# Run Pytorch CPU Predictions

In [35]:
import torch
from transformers import Wav2Vec2Processor,Wav2Vec2ForCTC
from convert_wav2vec2 import get_sample,get_inuputs_from_audio
import time 
import librosa

model = Wav2Vec2ForCTC.from_pretrained(model_id)
processor = Wav2Vec2Processor.from_pretrained(model_id)

def pytorch_asr(path,model,processor):
    # get inputs
    inputs = get_inuputs_from_audio(path=path, processor=processor, tensor_type="pt")
    # run inference
    st = time.time()
    logits = model(**inputs)[0]
    dur = time.time() - st
    # decode
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription, dur


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
print(f"running pytorch inference with {model_id} on CPU")
sample = get_sample(1)
print(f"Duration of {sample} is {librosa.get_duration(filename=sample)}s")

trans , duration = pytorch_asr(sample,model,processor)
print(f"Prediction with Pytorch took {round(duration,2)}s")
print(f"Meaining: 1 second audio takes {round(round(duration,2)/librosa.get_duration(filename=sample),2)} seconds to predict")
print(f"transcript: \n{trans}")

running pytorch inference with facebook/wav2vec2-base-960h on CPU
Duration of sample1.flac is 13.69s
Prediction with Pytorch took 1.3s
Meaining: 1 second audio takes 0.09 seconds to predict
transcript: 
GOING ALONG SLUSHY COUNTRY ROADS AND SPEAKING TO DAMP AUDIENCES IN DRAUGHTY SCHOOL ROOMS DAY AFTER DAY FOR A FORTNIGHT HE'LL HAVE TO PUT IN AN APPEARANCE AT SOME PLACE OF WORSHIP ON SUNDAY MORNING AND HE CAN COME TO US IMMEDIATELY AFTERWARDS


# GPU test (Not tested yet)

In [4]:
!pip uninstall onnxruntime -y
!pip install onnxruntime-gpu

In [ ]:
# create onnx modelsf
from convert_wav2vec2 import convert_wav2vec2_onnx

!rm -rf exports
model_id = "facebook/wav2vec2-base-960h"
convert_wav2vec2_onnx(model_id=model_id, optimize=True, quantize=False, use_gpu=True)

# Run ONNX GPU Predictions

In [ ]:
import onnxruntime as ort
import numpy as np 
from transformers import Wav2Vec2Processor
from convert_wav2vec2 import get_sample,get_inuputs_from_audio
import time 
import librosa

sess_options = ort.SessionOptions()

# Set graph optimization level
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL

model_path="exports/wav2vec2-base-960h-opt.onnx"
ort_session = ort.InferenceSession(model_path, sess_options,providers=['CUDAExecutionProvider'])
processor = Wav2Vec2Processor.from_pretrained(model_id)


def onnx_asr(path,sess,processor):
    # get inputs
    onnx_inputs = get_inuputs_from_audio(path=path, processor=processor, tensor_type="np")
    # run inference
    st = time.time()
    logits = sess.run(None, onnx_inputs.data)[0]
    dur = time.time() - st
    # decode
    predicted_ids=np.argmax(logits, axis=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription, dur
